In [4]:
from keras.models import load_model
import numpy as np
import pandas as pd
import csv

C:\Users\thetong\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [5]:
SET50 = ['ADVANC', 'AOT', 'BANPU', 'BBL', 'BCP', 'BDMS',
        'BEM', 'BH', 'BJC', 'BTS', 'CENTEL', 'CPALL', 'CPF',
        'CPN', 'DTAC', 'EGCO', 'HMPRO', 'INTUCH', 
        'IRPC', 'KBANK', 'KCE', 'KKP', 'KTB', 'LH', 'MINT', 
        'PTT', 'PTTEP', 'ROBINS', 'SCB', 'SCC', 
        'TCAP', 'TISCO', 'TMB', 'TOP', 'TRUE', 'TU']

In [7]:
def load_test_data(symbol, year_start, year_end):
    stock_data = []
    with open('./Data set/SET50_OHLC_FIXED/' + symbol + '.BK.csv', 'r') as csv_file:
        file_data = csv.reader(csv_file, delimiter=',')
        file_data = list(file_data)[1:]
        ma = []
        for row in file_data:
            if row[1] is '':
                continue
            elif int(row[0][0:4]) == year_start - 1:
                if len(ma) < 30:
                    ma.append([float(row[1]), float(row[2]), float(row[3]), float(row[4])])
                    continue
                temp = [float(row[1]), float(row[2]), float(row[3]), float(row[4])]
                stock_data.append(find_diff_ma(ma, temp))
                del ma[0]
                ma.append(temp)
                if len(stock_data) > 29:
                    del stock_data[0]
            elif int(row[0][0:4]) >= year_start and int(row[0][0:4]) <= year_end:
                temp = [float(row[1]), float(row[2]), float(row[3]), float(row[4])]
                stock_data.append(find_diff_ma(ma, temp))
                del ma[0]
                ma.append(temp)
    return stock_data

def find_diff_ma(moving_averages, current_values):
    returns = []
    for i in range(len(moving_averages[0])):
        data = [moving_average[i] for moving_average in moving_averages]
        ma = np.sum(data) / 30
        diff_ma = current_values[i] - ma
        returns.append(diff_ma)
    return returns
        

In [8]:
def load_stock_data(symbol, year_start, year_end):
    stock_data = []
    with open('./Data set/SET50_OHLC_FIXED/' + symbol + '.BK.csv', 'r') as csv_file:
        file_data = csv.reader(csv_file, delimiter=',')
        file_data = list(file_data)[1:]
        ma = []
        for row in file_data:
            if row[1] is '':
                continue
            elif len(ma) < 30:
                ma.append([float(row[1]), float(row[2]), float(row[3]), float(row[4])])
            elif int(row[0][0:4]) >= year_start and int(row[0][0:4]) <= year_end:
                temp = [float(row[1]), float(row[2]), float(row[3]), float(row[4])]
                stock_data.append(find_diff_ma(ma, temp))
                del ma[0]
                ma.append(temp)
    return stock_data

In [13]:
train = [ load_stock_data(SET50[i], 2008, 2014) for i in range(len(SET50))]
test = [ load_test_data(SET50[i], 2015, 2017) for i in range(len(SET50))]

In [14]:
train_length = len(train[0])
test_length = len(test[0])

In [15]:
train_length, test_length

(1685, 759)

In [18]:
data_train = []
index = 0
while index + 30 < train_length:
    temp = [i[index : index + 30] for i in train]
    data_train.append(temp)
    index += 7
    
x_train = np.asarray(data_train).astype('float32')
x_lstm_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], x_train.shape[2] * x_train.shape[3]))
x_cnnlstm_train = np.reshape(x_train, (x_train.shape[0], 1, x_train.shape[1], x_train.shape[2] , x_train.shape[3]))

print(x_train.shape, x_lstm_train.shape, x_cnnlstm_train.shape)

(237, 36, 30, 4) (237, 36, 120) (237, 1, 36, 30, 4)


In [19]:
data_test = []
index = 0
while index + 30 < test_length:
    temp = [i[index : index + 30] for i in test]
    data_test.append(temp)
    index += 7
    
x_test = np.asarray(data_test).astype('float32')
x_lstm_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], x_test.shape[2] * x_test.shape[3]))
x_cnnlstm_test = np.reshape(x_test, (x_test.shape[0], 1, x_test.shape[1], x_test.shape[2] , x_test.shape[3]))

print(x_test.shape, x_lstm_test.shape, x_cnnlstm_test.shape)

(105, 36, 30, 4) (105, 36, 120) (105, 1, 36, 30, 4)


In [20]:
def predict_every_30_days(x, model):
    predicts = []
    for (index, element) in enumerate(x):
        data = np.asarray([element])
        for i in range(30):
            predict = model.predict(data, verbose = 0)
            predicts.append(np.asarray(predict[0]).T)
            data = find_new_data(data, predict[0])
    return predicts
    
def find_new_data(x, predict):
    for (index, element) in enumerate(x[0]):
        for i in range(len(element)):
            if i == 0:
                continue
            elif i < 29:
                element[i - 1] = element[i]
            else:
                element[i - 1] = element[i]
                element[i] = predict[index]
    return x

### CNN PREDICTION

In [21]:
cnn_model = load_model('./models/CNNMAinput4output1.h5')

In [22]:
cnn_predictions = predict_every_30_days(x_train, cnn_model)
cnn_predictions_test = predict_every_30_days(x_test, cnn_model)

In [23]:
cnn_predictions = np.asarray(cnn_predictions).T
np.savetxt('cnn7Predictions.txt', cnn_predictions, fmt='%f')
cnn_predictions_test = np.asarray(cnn_predictions_test).T
np.savetxt('cnn7TestPredictions.txt', cnn_predictions_test, fmt='%f')

### LSTM PREDICTION

In [24]:
lstm_model = load_model('./models/LSTM_ma_4in1out.h5')

In [ ]:
lstm_predictions = predict_every_30_days(x_lstm_train, lstm_model)
lstm_predictions_test = predict_every_30_days(x_lstm_test, lstm_model)

In [14]:
lstm_predictions = np.asarray(lstm_predictions).T
np.savetxt('lstm7Predictions.txt', lstm_predictions_test, fmt='%f')
lstm_predictions_test = np.asarray(lstm_predictions_test).T
np.savetxt('lstm7TestPredictions.txt', lstm_predictions_test, fmt='%f')

### CNN LSTM PREDICTION

In [15]:
cnnlstm_model = load_model('./models/cnnlstmMA4Input1Output.h5')

In [ ]:
cnnlstm_predictions = predict_every_30_days(x_cnnlstm_train, cnnlstm_model)
cnnlstm_predictions_test = predict_every_30_days(x_cnnlstm_test, cnnlstm_model)

In [ ]:
cnnlstm_predictions = np.asarray(cnnlstm_predictions).T
np.savetxt('cnnlstm7Predictions.txt', cnnlstm_predictions_test, fmt='%f')
cnnlstm_predictions_test = np.asarray(cnnlstm_predictions_test).T
np.savetxt('cnnlstm7TestPredictions.txt', cnnlstm_predictions_test, fmt='%f')